In [1]:
import unet
import losses
import engine
import torch
import json
import unet2d 
from torch import nn


class NeuralNetwork(unet2d.UNET,engine.Engine):
    def __init__(self):
        engine.Engine.__init__(self)
        unet2d.UNET.__init__(self)


with open('../config/configs.json') as f:
    config = json.load(f)

model=NeuralNetwork()
model.compile(loss= nn.BCEWithLogitsLoss(), optimizer= torch.optim.Adam(model.parameters(),lr=config['optimizer']['learning_rate']), metrics=['dice_score','loss'])
print("Compile Status")
print("-------------------------------")
# model.compile_status()

model.load_data(dataset_path=config['dataset']['Barbary'],transformation_flag=True,transformation=config['transformation_size'],batchsize=config['batch_size'],test_valid_split=0)
print("\nData Status")
print("-------------------------------")
model.data_status()

print("\nTesting After Random Initialization")
print("-------------------------------")
model.evaluate_train2d()

print("\nLoading Pretrained Model")
print("-------------------------------")
model.load_checkpoint(config['checkpoint'])

print("\nTesting After Loading Pretrained Weights")
print("-------------------------------")
model.evaluate_train2d()

print("\nTraining")
print("-------------------------------")
model.fit2d(epochs=config['epochs'])

print("\nTesting After Training")
print("-------------------------------")
model.evaluate_train2d()

c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Compile Status
-------------------------------

Data Status
-------------------------------
Batch Shape of Training Features: (1, 1, 64, 64, 64) torch.float32
Batch Shape of Training Labels: (1, 1, 64, 64, 64) torch.float32

Testing After Random Initialization
-------------------------------
batch: 1/5
batch: 2/5
batch: 3/5
batch: 4/5
batch: 5/5
Total Loss: 0.6442962050437927

Loading Pretrained Model
-------------------------------
=> Loading checkpoint

Testing After Loading Pretrained Weights
-------------------------------
batch: 1/5
batch: 2/5
batch: 3/5
batch: 4/5
batch: 5/5
Total Loss: 0.16834119856357574

Training
-------------------------------
Epoch 1
-------------------------------
batch: 1/5
batch: 2/5
batch: 3/5
batch: 4/5
batch: 5/5
3D Loss: 0.1162098228931427

Testing After Training
-------------------------------
batch: 1/5
batch: 2/5
batch: 3/5
batch: 4/5
batch: 5/5
Total Loss: 0.1271007925271988


In [6]:
import torch

print("\nPrediction")
print("-------------------------------")
ImagePath=config['PredictPath']
# Image=torch.asarray(nib.load(ImagePath).get_fdata())
output=model.pred2d(ImagePath)
print(output.shape)


Prediction
-------------------------------
batch: 1/1
(1, 1, 1, 64, 64, 64)


In [7]:
import numpy as np
import nibabel as nib

ni_img = nib.Nifti1Image(np.asarray(output[0][0][0]), affine=np.eye(4))
nib.save(ni_img, "nifti_prediction.nii")

In [8]:
#if model is better uncomment this line
# model.save_checkpoint(config['checkpoint'])

=> Saving checkpoint
